In [1]:
import torch
import torch.nn as nn
from PIL import Image
import numpy as np

In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
import zipfile
with zipfile.ZipFile('/content/sample_data/data.zip', 'r') as zip_ref:
  zip_ref.extractall('dataset')

In [4]:
import torch
import torch.nn as nn

class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Conv layer
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=3
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


#### **Question 1: Which loss function you will use?**

In [5]:
model = MyCNN()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)

criterion = nn.BCEWithLogitsLoss()

#### **Question 2: What's the total number of parameters of the model?**

In [6]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


In [7]:
from torchvision import transforms

In [8]:
import os
from torch.utils.data import Dataset

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [10]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='/content/dataset/data/train',
    transform=train_transforms
)

val_dataset = HairDataset(
    data_dir='/content/dataset/data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6320, Acc: 0.6467, Val Loss: 1.1013, Val Acc: 0.5025
Epoch 2/10, Loss: 0.7225, Acc: 0.5855, Val Loss: 0.6660, Val Acc: 0.5622
Epoch 3/10, Loss: 0.6233, Acc: 0.6492, Val Loss: 0.6092, Val Acc: 0.6667
Epoch 4/10, Loss: 0.5230, Acc: 0.7278, Val Loss: 0.7362, Val Acc: 0.6219
Epoch 5/10, Loss: 0.5199, Acc: 0.7615, Val Loss: 0.6584, Val Acc: 0.6567
Epoch 6/10, Loss: 0.4222, Acc: 0.8002, Val Loss: 0.6744, Val Acc: 0.6667
Epoch 7/10, Loss: 0.3745, Acc: 0.8340, Val Loss: 0.9187, Val Acc: 0.6119
Epoch 8/10, Loss: 0.4113, Acc: 0.8152, Val Loss: 0.7447, Val Acc: 0.6418
Epoch 9/10, Loss: 0.2651, Acc: 0.8914, Val Loss: 0.6999, Val Acc: 0.6567
Epoch 10/10, Loss: 0.2224, Acc: 0.9114, Val Loss: 1.0841, Val Acc: 0.6169


#### **Question 3: What is the median of training accuracy for all the epochs for this model?**

In [13]:
np.median(history['acc'])

np.float64(0.7808988764044944)

#### **Question 4: What is the standard deviation of training loss for all the epochs for this model?**

In [14]:
np.std(history['loss'])

np.float64(0.15391155320575076)

In [15]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [16]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.1988, Acc: 0.9151, Val Loss: 0.9404, Val Acc: 0.7214
Epoch 2/10, Loss: 0.1559, Acc: 0.9426, Val Loss: 0.7331, Val Acc: 0.7164
Epoch 3/10, Loss: 0.0840, Acc: 0.9813, Val Loss: 0.8528, Val Acc: 0.7313
Epoch 4/10, Loss: 0.0859, Acc: 0.9750, Val Loss: 1.0475, Val Acc: 0.7214
Epoch 5/10, Loss: 0.1896, Acc: 0.9263, Val Loss: 0.9096, Val Acc: 0.7164
Epoch 6/10, Loss: 0.0503, Acc: 0.9938, Val Loss: 0.8527, Val Acc: 0.7512
Epoch 7/10, Loss: 0.0252, Acc: 0.9988, Val Loss: 0.9411, Val Acc: 0.7065
Epoch 8/10, Loss: 0.0417, Acc: 0.9938, Val Loss: 0.9967, Val Acc: 0.7264
Epoch 9/10, Loss: 0.0136, Acc: 1.0000, Val Loss: 0.9915, Val Acc: 0.7363
Epoch 10/10, Loss: 0.0084, Acc: 1.0000, Val Loss: 1.0508, Val Acc: 0.7214


#### **Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?**

In [17]:
np.mean(history['val_loss'])

np.float64(0.9316075529279395)

#### **Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?**

In [19]:
avg= (0.7512 + 0.7065 + 0.7264 + 0.7363 + 0.7214) / 5
avg

0.72836